In [ ]:
# draw function on 2d elements

import webgpu.jupyter as wj
from ngsolve import *
from ngsolve_webgpu.mesh import MeshData, PointNumbersRenderObject
from ngsolve_webgpu.cf import FunctionData, CoefficientFunctionRenderObject

def draw_func_with_vert_numbers(mesh, cf, order=2):
    mesh_data = MeshData(mesh.ngmesh)
    function_data = FunctionData(mesh_data, cf, order)
    mesh_object = CoefficientFunctionRenderObject(function_data)
    point_numbers = PointNumbersRenderObject(mesh_data, 13)
    mesh_object.colormap.set_min_max(0., 1.)
    wj.Draw([mesh_object.colormap, mesh_object, point_numbers])
    
m = Mesh(unit_square.GenerateMesh(maxh=.1))
cf=sin(10*x)*sin(10*y)

draw_func_with_vert_numbers(m, cf, order=5)


In [ ]:
# draw geometry

from netgen.occ import *
from ngsolve import *
import webgpu.jupyter

box = Box((-1,-1,-1), (1,1,1)) + Sphere((1,1,1), 0.4)

def draw_geo(geo):
    import ngsolve_webgpu.geometry
    if isinstance(geo, TopoDS_Shape):
        geo = OCCGeometry(geo)
    obj = ngsolve_webgpu.geometry.GeometryRenderObject(geo)
    wj.Draw([obj])

draw_geo(box)

In [ ]:
# draw isosurface
import webgpu.jupyter
from netgen.occ import *
from ngsolve import *
import ngsolve_webgpu.cf
import numpy as np
import importlib
importlib.reload(ngsolve_webgpu.cf)

box = Box((-1,-1,-1), (1,1,1))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=.2))
gf = GridFunction(H1(mesh, order=1))
cf = x**2 + y**2 + z**2 - 1.2**2
gf.Set(cf)

def draw_iso(levelset, func, mesh):
    import ngsolve_webgpu.isosurface
    import ngsolve_webgpu.mesh
    import ngsolve_webgpu.cf
    meshdata = ngsolve_webgpu.mesh.MeshData(mesh.ngmesh)
    data = ngsolve_webgpu.cf.FunctionData(meshdata, func, order=1)
    levelset_data = ngsolve_webgpu.cf.FunctionData(meshdata, -levelset, order=1)
    neg_outer = ngsolve_webgpu.isosurface.NegativeSurfaceRenderer(data, levelset_data)
    clipping = neg_outer.clipping
    c = np.array(clipping.center)
    n = np.array(clipping.normal)
    n /= np.linalg.norm(n)
    cp_parameters = [Parameter(pi) for pi in c] + [Parameter(-np.dot(c,n))]
    iso = ngsolve_webgpu.isosurface.IsoSurfaceRenderObject(-levelset, func, mesh, "Isosurface")
    clipping.mode = clipping.Mode.PLANE
    iso.colormap = neg_outer.colormap
    iso.colormap.set_min_max(-1, 2)
    iso.clipping = clipping
    # scene = webgpu.jupyter.Scene([neg_outer.colormap, iso, neg_outer, clipping])
    scene = webgpu.jupyter.Scene([neg_outer.colormap, iso, clipping, neg_outer])
    webgpu.jupyter.Draw(scene)
    clipping.add_options_to_gui(scene.gui)
    return scene

scene = draw_iso(cf, x, mesh)

In [ ]:
gf.Set(x**2 + y**2 + z**2 - 1.1**2)
scene.redraw()